In [1]:
!pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 2.2 MB/s eta 0:00:0000:0100:01


In [18]:
from kafka import KafkaProducer
import csv
import json

producer = KafkaProducer(bootstrap_servers="localhost:9092", value_serializer=lambda v: json.dumps(v).encode('utf-8'))
i = 0
with open("./dataset.csv") as f:
    reader = csv.DictReader(f)
    for row in reader:
        i = i + 1
        producer.send(topic="mytopic", value=row)
        producer.flush()
        if i == 10:
            break

In [19]:
from kafka import KafkaConsumer

consumer = KafkaConsumer("mytopic", bootstrap_servers="localhost:9092", auto_offset_reset='earliest', consumer_timeout_ms=10000)
data = []
for message in consumer:
    data.append(json.loads(message.value))
consumer.close()

with open("output_dataset.json", "w+") as final:
    json.dump(data, final)

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ProiectIBD').getOrCreate()

In [6]:
df = spark.read.option("header", "True").csv("./dataset.csv")
df.show(5)

+----------+-----------+-----+--------------------+-----+---------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------+----------+----------+------------------+-------------------+--------------------+
|ARREST_KEY|ARREST_DATE|PD_CD|             PD_DESC|KY_CD|      OFNS_DESC|  LAW_CODE|LAW_CAT_CD|ARREST_BORO|ARREST_PRECINCT|JURISDICTION_CODE|AGE_GROUP|PERP_SEX|     PERP_RACE|X_COORD_CD|Y_COORD_CD|          Latitude|          Longitude|             Lon_Lat|
+----------+-----------+-----+--------------------+-----+---------------+----------+----------+-----------+---------------+-----------------+---------+--------+--------------+----------+----------+------------------+-------------------+--------------------+
| 236791704| 11/22/2021|  581|                NULL| NULL|           NULL|PL 2225001|         M|          M|             28|                0|    45-64|       M|         BLACK|    997427|    230378|40.799008797000056| -73.95240